In [ ]:
import RPi.GPIO as GPIO
import cv2
from time import sleep
from gpiozero import Servo
from gpiozero.pins.pigpio import PiGPIOFactory
GPIO.setmode(GPIO.BOARD)
factory = PiGPIOFactory()
s1 = Servo(13, min_pulse_width=0.5/1000, max_pulse_width=2.5/1000, pin_factory=factory)
s2 = Servo(12, min_pulse_width=0.5/1000, max_pulse_width=2.5/1000, pin_factory=factory)
GPIO.setup(7, GPIO.OUT)
GPIO.setup(11, GPIO.OUT)
t1 = GPIO.PWM(7, 50)
t2 = GPIO.PWM(11, 50)
t1.start(0)
t2.start(0)

def arm_open():
    s1.max()
    s2.min()
    sleep(3)
def arm_mid():
    s1.mid()
    s2.mid()
    sleep(3)
def arm_close():
    s1.min()
    s2.max()
    sleep(3)

def forward():
    print('forward')
    t1.ChangeDutyCycle(7)
    t2.ChangeDutyCycle(7)
def right():
    print('right')
    t1.ChangeDutyCycle(3)
    t2.ChangeDutyCycle(5)
def left():
    print('left')
    t1.ChangeDutyCycle(5)
    t2.ChangeDutyCycle(3)
def stop():
    print('stop')
    t2.stop()
    t1.stop()
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
while True:
    _,img = cap.read()
    rows, cols, _ = img.shape
    center = int(cols/2)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y),(x+w, y+h), (0, 255, 255), 2)
        length = (x+w)-x
        print(length)
        x_medium = int((x+x+w)/2)
        cv2.line(img, (x_medium, 0), (x_medium, 480), (255, 0, 0), 2)
        w = w/4
        if x_medium>center+30:
            left()
        if x_medium<center+30:
            right()
        if w < 100:
            forward()
            arm_open()
        else:
            stop()
            arm_mid()
        cv2.putText(img, str(w), (x,y),  cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), thickness=4)
    cv2.line(img, (center, 0), (center, 480), (0, 255, 0), 2)
    cv2.imshow("img", img)
    if cv2.waitKey(1) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()
